In [13]:
import pandas as pd
from sklearn import datasets # sklearn comes with some toy datasets to practise
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score


import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = datasets.less_features_df()


X = pd.DataFrame(data["data"], columns=data["feature_names"])

#print()
X.head()

In [ ]:
kmeans = KMeans(n_clusters=8, random_state=1234)
kmeans.fit(X_scaled_df)

In [6]:
def kmeans_clustering(audio_features_df, num_clusters, audio_feature_columns):
    # Select the columns for clustering
    data = audio_features_df[audio_feature_columns]

    # Create a K-Means model
    kmeans = KMeans(n_clusters=num_clusters, random_state=0)

    # Fit the K-Means model to the data
    kmeans.fit(data)

    # Assign cluster labels to the original DataFrame
    audio_features_df['cluster_label'] = kmeans.labels_

    # Calculate the Calinski-Harabasz score
    calinski_score = calinski_harabasz_score(data, kmeans.labels_)

    return audio_features_df, calinski_score


In [7]:
songs_df=pd.read_csv('clustered_songs.csv')

In [8]:
songs_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,cluster
0,0.838,0.626,0,-4.214,1,0.0396,0.00211,0.000059,0.0221,0.108,97.994,163514,4,3
1,0.611,0.478,0,-5.839,1,0.1070,0.03500,0.000000,0.1070,0.138,172.076,258158,4,0
2,0.744,0.697,11,-5.063,0,0.1970,0.01060,0.000000,0.0842,0.767,149.280,247413,4,2
3,0.823,0.703,8,-2.691,1,0.2980,0.01610,0.000001,0.2940,0.260,178.043,194214,4,2
4,0.649,0.200,2,-19.410,1,0.0507,0.83900,0.000306,0.2060,0.866,150.043,160325,4,4


In [9]:
songs_df.shape

(5232, 14)

In [10]:
# Assuming audio_features_df is your audio features DataFrame and audio_feature_columns is a list of column names for clustering.
num_clusters = 3  # Set the number of clusters you want
audio_feature_columns=["danceability","energy"]
clustered_df, calinski_score = kmeans_clustering(songs_df, num_clusters, audio_feature_columns)

C:\Users\i.glavatskyi\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\i.glavatskyi\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\i.glavatskyi\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\i.glavatskyi\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:757: FutureWarning: is_sparse is deprecated and will be

In [11]:
clustered_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,cluster,cluster_label
0,0.838,0.626,0,-4.214,1,0.0396,0.00211,0.000059,0.0221,0.108,97.994,163514,4,3,2
1,0.611,0.478,0,-5.839,1,0.1070,0.03500,0.000000,0.1070,0.138,172.076,258158,4,0,1
2,0.744,0.697,11,-5.063,0,0.1970,0.01060,0.000000,0.0842,0.767,149.280,247413,4,2,2
3,0.823,0.703,8,-2.691,1,0.2980,0.01610,0.000001,0.2940,0.260,178.043,194214,4,2,2
4,0.649,0.200,2,-19.410,1,0.0507,0.83900,0.000306,0.2060,0.866,150.043,160325,4,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5227,0.475,0.789,1,-6.124,1,0.0380,0.17000,0.000000,0.1470,0.485,155.227,226091,4,0,0
5228,0.751,0.256,0,-11.702,1,0.0835,0.02570,0.579000,0.0958,0.609,172.180,168837,4,3,1
5229,0.663,0.502,5,-7.070,1,0.0285,0.79100,0.000000,0.1050,0.371,143.969,183087,4,4,1
5230,0.864,0.578,1,-8.310,1,0.4460,0.04400,0.000000,0.0827,0.476,161.903,204074,4,3,2


In [12]:
calinski_score

3681.731674202956